In [94]:
import pandas as pd
import numpy as np
from nltk import *
import re

In [95]:
df = pd.read_csv('tmdb_movies_data.csv')
#df.head()

In [96]:
# df.info()

In [97]:
# df.columns

In [98]:
df = df[['popularity', 'budget', 'revenue', 'original_title',
       'cast', 'director', 'keywords', 'overview', 'genres', 'vote_average', 'imdb_id']]

In [99]:
df['net_profit'] = df['revenue'] - df['budget']
df = df.drop(['budget', 'revenue'], axis=1)

In [100]:
def verticalBar_remover(df, col_name):
    new = []
    
    for item in df[col_name]:
        if type(item) != str:
            item = str(item)
        y = ''
        for i in item.split('|'):
            y = y + ' ' + i   #y = y + ',' + ' ' + i
                 
        new.append(y.strip())
    #print(new)
    df[col_name] = new


In [101]:
clean_df = df
for col_name in ['cast','director', 'keywords', 'genres']:
    verticalBar_remover(clean_df, col_name)

In [102]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [103]:
from nltk.corpus import stopwords 

def tokeniser(df, col):
    sw = list(set(stopwords.words('english'))) + ['.', ',', '?']
    li = []
    for sentt in df[col]:
        words = word_tokenize(str(sentt).lower())
        words = [w for w in words if w not in sw]
        final = ''
        for x in words:
            final = final + " " + x
        li.append(final)
    df[col] = li

tokeniser(clean_df, 'overview')

In [104]:
def numbers_normalised(df, col_list):
    for col in col_list:
        maxx = df[col].max()
        minn = df[col].min()
        df[col] = round((df[col] - minn) / (maxx - minn), 3) 
    return df.head()

numbers_normalised(clean_df, ['popularity', 'vote_average', 'net_profit' ])

,popularity,original_title,cast,director,keywords,overview,genres,vote_average,imdb_id,net_profit
0,1.000,Jurassic World,Chris Pratt Bryce Dallas Howard Irrfan Khan Vi...,Colin Trevorrow,monster dna tyrannosaurus rex velociraptor island,twenty-two years events jurassic park isla nu...,Action Adventure Science Fiction Thriller,0.649,tt0369610,0.601
1,0.862,Mad Max: Fury Road,Tom Hardy Charlize Theron Hugh Keays-Byrne Nic...,George Miller,future chase post-apocalyptic dystopia australia,apocalyptic story set furthest reaches planet...,Action Adventure Science Fiction Thriller,0.727,tt1392190,0.217
2,0.398,Insurgent,Shailene Woodley Theo James Kate Winslet Ansel...,Robert Schwentke,based on novel revolution dystopia sequel dyst...,beatrice prior must confront inner demons con...,Adventure Science Fiction Thriller,0.623,tt2908446,0.203
3,0.339,Star Wars: The Force Awakens,Harrison Ford Mark Hamill Carrie Fisher Adam D...,J.J. Abrams,android spaceship jedi space opera 3d,thirty years defeating galactic empire han so...,Action Adventure Science Fiction Fantasy,0.779,tt2488496,0.771
4,0.283,Furious 7,Vin Diesel Paul Walker Jason Statham Michelle ...,James Wan,car race speed revenge suspense car,deckard shaw seeks revenge dominic toretto fa...,Action Crime Thriller,0.753,tt2820852,0.585


In [121]:
clean_df['final_numeric_weights'] = clean_df['popularity'] + clean_df['vote_average'] + clean_df['net_profit']

In [122]:
# Left out overview from dataframe
textual_df = clean_df[['original_title' , 'cast', 'director','keywords', 'genres', 'imdb_id']].apply(lambda x: x.astype(str).str.lower())
numeric_df = clean_df['final_numeric_weights']
#type(numeric_df['net_profit'][0])
final_df = textual_df.merge(numeric_df, left_index=True, right_index=True)
# final_df['text'] = final_df['cast'] + final_df['director'] + final_df['genres'] + final_df['keywords']
# final_df = final_df[['text', 'popularity', 'vote_average', 'net_profit' ]]
final_df.head()

,original_title,cast,director,keywords,genres,imdb_id,final_numeric_weights
0,jurassic world,chris pratt bryce dallas howard irrfan khan vi...,colin trevorrow,monster dna tyrannosaurus rex velociraptor island,action adventure science fiction thriller,tt0369610,2.250
1,mad max: fury road,tom hardy charlize theron hugh keays-byrne nic...,george miller,future chase post-apocalyptic dystopia australia,action adventure science fiction thriller,tt1392190,1.806
2,insurgent,shailene woodley theo james kate winslet ansel...,robert schwentke,based on novel revolution dystopia sequel dyst...,adventure science fiction thriller,tt2908446,1.224
3,star wars: the force awakens,harrison ford mark hamill carrie fisher adam d...,j.j. abrams,android spaceship jedi space opera 3d,action adventure science fiction fantasy,tt2488496,1.889
4,furious 7,vin diesel paul walker jason statham michelle ...,james wan,car race speed revenge suspense car,action crime thriller,tt2820852,1.621


In [47]:
# All textual data in one col

# df2 = final_df['text']
# cv = CountVectorizer(ngram_range=(1,3))
# text_vectorised = cv.fit_transform(df2).toarray()
# cv.vocabulary_

# Category wise vectorised

In [106]:
#Genre Vectoriser
cv = CountVectorizer(ngram_range=(1,1))
genre_vectorised = cv.fit_transform(final_df['genres']).toarray()
# Director Vectoriser
cv2 = CountVectorizer(ngram_range=(2,3))
director_vectorised = cv.fit_transform(final_df['director']).toarray()
# Cast Vectoriser
cv3 = CountVectorizer(ngram_range=(2,3))
cast_vectorised = cv.fit_transform(final_df['cast']).toarray()

#Cosine Similarities
cos_sim_genre = cosine_similarity(genre_vectorised, genre_vectorised)
cos_sim_director = cosine_similarity(director_vectorised, director_vectorised)
cos_sim_cast = cosine_similarity(cast_vectorised, cast_vectorised)

In [49]:
#final_df[final_df['original_title'] == '3 idiots']
# list(final_df[final_df['director'] == 'colin trevorrow']['original_title'])

,original_title,cast,director,keywords,genres,imdb_id,popularity,vote_average,net_profit
1486,3 idiots,aamir khan kareena kapoor madhavan sharman jos...,rajkumar hirani,suicide suicide attempt college friends engine...,drama comedy romance,tt1187043,0.816681,7.8,61000000


# Adding weight of Numerical data like profit etc to cosine similarity to get an overall weight involving all factors

In [128]:
idx =  [0, 1 , 2]
final_df['final_numeric_weights'].loc[idx]

0    2.250
1    1.806
2    1.224
Name: final_numeric_weights, dtype: float64

In [200]:
  def recommendation(movies):
    
    idx = final_df[final_df['original_title'] == movies ].index
    
    casts = pd.Series(cos_sim_cast[idx][0]).sort_values(ascending=False)[:10]
    cast_idx = casts.index 
    cast_recom_idx = (casts + final_df['final_numeric_weights'].loc[cast_idx]).sort_values(ascending=False)
    #cast_recom = final_df['original_title'].loc[cast_recom_idx]
    
    genres = pd.Series(cos_sim_genre[idx][0]).sort_values(ascending=False)[:10]
    genre_idx = genres.index
    genre_recom = final_df['original_title'].loc[genre_idx]
    
    directors = pd.Series(cos_sim_director[idx][0]).sort_values(ascending=False)[:10]
    director_idx = directors.index 
    director_recom_idx = (directors + final_df['final_numeric_weights'].loc[director_idx]).sort_values(ascending=False)
    director_recom = final_df['original_title'].loc[director_recom_idx]
    
    final = pd.Series(list(cast_recom_idx) + list(director_recom_idx) + list(genre_idx)).sort_values(ascending=False).index
    print(final_df['original_title'].loc[final][:10])
    
recommendation('3 idiots')

KeyError: "None of [Float64Index([2.0039999999999996, 1.9979999999999998,              1.185,\n                           1.049,              0.812,              0.724,\n                           0.723,              0.685,              0.685,\n                           0.528],\n             dtype='float64')] are in the [index]"

In [183]:
final_df[final_df['original_title'] == 'the notebook']

,original_title,cast,director,keywords,genres,imdb_id,final_numeric_weights
6966,the notebook,rachel mcadams ryan gosling gena rowlands jame...,nick cassavetes,poem river sadness fight love of one's life,romance drama,tt0332280,1.052


In [184]:
final_df[final_df['original_title'] == 'jurassic world']

,original_title,cast,director,keywords,genres,imdb_id,final_numeric_weights
0,jurassic world,chris pratt bryce dallas howard irrfan khan vi...,colin trevorrow,monster dna tyrannosaurus rex velociraptor island,action adventure science fiction thriller,tt0369610,2.25
